Tensorflow 1.X

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
in_a = tf.placeholder(dtype=tf.float32, shape=(2))
def model(x):
    with tf.variable_scope("matmul"):
        W = tf.get_variable("W", initializer=tf.ones(shape=(2,2)))
        b = tf.get_variable("b", initializer=tf.zeros(shape=(2)))
    return x * W + b
out_a = model(in_a)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    outs = sess.run([out_a],
                feed_dict={in_a: [1, 0]})
    writer = tf.summary.FileWriter("./logs/example", sess.graph)

Instructions for updating:
non-resource variables are not supported in the long term


Tensorflow 2.x

In [9]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
# Network and training parameters.
EPOCHS = 10
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10 # number of outputs = number of digits
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2 # how much TRAIN is reserved for VALIDATION

# Loading MNIST dataset.
# verify
# You can verify that the split between train and test is 60,000, and 10,000 respectively.
# Labels have one-hot representation.is automatically applied
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
# X_train is 60000 rows of 28x28 values; we --> reshape it to
# 60000 x 784.
RESHAPED = 784
#
X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# Normalize inputs to be within in [0, 1].
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
# One-hot representation of the labels.
Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES)
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)


# Build the model.
model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(NB_CLASSES,
input_shape=(RESHAPED,),
name='dense_layer',
activation='softmax'))

# Compiling the model.
model.compile(optimizer='SGD',
loss='categorical_crossentropy',
metrics=['accuracy'])

# Training the model.
model.fit(X_train, Y_train,
batch_size=BATCH_SIZE, epochs=EPOCHS,
verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

#evaluate the model
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)

60000 train samples
10000 test samples
Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 4s 77us/sample - loss: 1.3835 - acc: 0.6578 - val_loss: 0.8977 - val_acc: 0.8243
Epoch 2/10
48000/48000 [==============================] - 3s 69us/sample - loss: 0.7955 - acc: 0.8262 - val_loss: 0.6589 - val_acc: 0.8574
Epoch 3/10
48000/48000 [==============================] - 4s 74us/sample - loss: 0.6447 - acc: 0.8491 - val_loss: 0.5630 - val_acc: 0.8690
Epoch 4/10
48000/48000 [==============================] - 4s 79us/sample - loss: 0.5718 - acc: 0.8607 - val_loss: 0.5096 - val_acc: 0.8778
Epoch 5/10
48000/48000 [==============================] - 4s 75us/sample - loss: 0.5274 - acc: 0.8684 - val_loss: 0.4751 - val_acc: 0.8833
Epoch 6/10
48000/48000 [==============================] - 3s 71us/sample - loss: 0.4969 - acc: 0.8737 - val_loss: 0.4507 - val_acc: 0.8877
Epoch 7/10
48000/48000 [==============================] - 4s 73us/sample - lo

Regression

In [ ]:
import tensorflow as tf
import pandas as pd
import tensorflow.feature_column as fc
from tensorflow.keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

features = ['CRIM', 'ZN',
'INDUS','CHAS','NOX','RM','AGE',
'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
x_train_df = pd.DataFrame(x_train, columns= features)
x_test_df = pd.DataFrame(x_test, columns= features)
y_train_df = pd.DataFrame(y_train, columns=['MEDV'])
y_test_df = pd.DataFrame(y_test, columns=['MEDV'])
x_train_df.head()

feature_columns = []
for feature_name in features:
    feature_columns.append(fc.numeric_column(feature_name,
    dtype=tf.float32))

def estimator_input_fn(df_data, df_label, epochs=10, shuffle=True,batch_size=32):
    def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(df_data), df_label))
        if shuffle:
            ds = ds.shuffle(100)
        ds = ds.batch(batch_size).repeat(epochs)
        return ds
    return input_function

train_input_fn = estimator_input_fn(x_train_df, y_train_df)
val_input_fn = estimator_input_fn(x_test_df, y_test_df, epochs=1,shuffle=False)

linear_est = tf.estimator.LinearRegressor(feature_columns=feature_
columns)
linear_est.train(train_input_fn, steps=100)
result = linear_est.evaluate(val_input_fn)

result = linear_est.predict(val_input_fn)
for pred,exp in zip(result, y_test[:32]):
print("Predicted Value: ", pred['predictions'][0], "Expected:
", exp)

In [19]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

# Load training and eval data
((train_data, train_labels),
(eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()

train_data = train_data/np.float32(255)
train_labels = train_labels.astype(np.int32)
eval_data = eval_data/np.float32(255)
eval_labels = eval_labels.astype(np.int32)

feature_columns = [tf.feature_column.numeric_column("x",shape=[28, 28])]

classifier = tf.estimator.LinearClassifier(
feature_columns=feature_columns,
n_classes=10,
model_dir="mnist_model/"
)

train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
x={"x": train_data},
y=train_labels,
batch_size=100,
num_epochs=None,
shuffle=True)

classifier.train(input_fn=train_input_fn, steps=100)

val_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
x={"x": eval_data},
y=eval_labels,
num_epochs=1,
shuffle=False)

eval_results = classifier.evaluate(input_fn=val_input_fn)
print(eval_results)

2.0.0
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'mnist_model/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000214E2B583C8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow: